# Задание 1

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Создаем сессию Spark
spark = SparkSession.builder.appName("SessionLength").getOrCreate()

df = spark.read.csv('data.csv', header=True, inferSchema=True)
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [22]:
df.show()

+---+----------+
| id| timestamp|
+---+----------+
|  1|1562007679|
|  1|1562007710|
|  1|1562007720|
|  1|1562007750|
|  2|1564682430|
|  2|1564682450|
|  2|1564682480|
+---+----------+



In [23]:
# Добавляем столбец с датой
df_with_date = df.withColumn("date", F.to_date(F.from_unixtime("timestamp")))

# Определяем окно для каждого пользователя
window_spec = Window().partitionBy("id", "date").orderBy("timestamp")

# Добавляем столбец с разницей между первым и последним действием
df_with_duration = df_with_date.withColumn("session_duration", F.last("timestamp").over(window_spec) - F.first("timestamp").over(window_spec))

# Группируем по id и рассчитываем усредненную длину сессии
result = df_with_duration.groupBy("id").agg(F.avg("session_duration").alias("avg_session_length"))

# Показываем результат
try:
    result.show()
except Exception as e:
    print(e)
    import traceback
    traceback.print_exc()


+---+------------------+
| id|avg_session_length|
+---+------------------+
|  1|             35.75|
|  2|23.333333333333332|
+---+------------------+



# Задание 2

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Создаем Spark Session
spark = SparkSession.builder.appName("AppName").getOrCreate()

# Загружаем данные о среднедневном спросе
df_dem = spark.read.csv("data_2.csv", header=True, inferSchema=True)

# Загружаем данные о запасах
df_stk = spark.read.csv("data_2_1.csv", header=True, inferSchema=True)

# calendar_df = spark.read.csv("data_2_2.csv", header=True, inferSchema=True)

# Объединяем данные с помощью join
df = df_dem.join(df_stk, ["product", "location"]).select(
    df_dem["product"].alias("prod"),
    df_dem["location"].alias("loc"),
    df_dem["demand"],
    df_stk["stock"]
)


In [25]:

# Рассчитываем количество дней в технических неделях
import calendar
from itertools import accumulate


inp_year = 2023  # Замените на ваш год
inp_month = 8    # Замените на ваш месяц
d_1, d_tot = calendar.monthrange(inp_year, inp_month)
week_len = [7 - d_1] + [7]*((d_1 + d_tot - 7)//7) + [(d_1 + d_tot) % 7]
if week_len[-1] == 0: del week_len[-1]
week_len_run = [0] + list(accumulate(week_len))  # нарастающий итог
print("Продолжительность технических недель:", week_len)

# Добавляем столбцы
def add_l(pf, n): return "wk" + str(n) + pf
new_cols_dict = dict(zip(
    [ add_l("-dem", n) for n in range(1, len(week_len) + 1) ] +              # заголовки - спрос
    ["op-stk"] + [ add_l("-stk", n) for n in range(1, len(week_len) + 1) ],  # заголовки - запас
    [ df['demand'] * wl for wl in week_len ] +                             # значения - спрос
    [ df['stock'] - df['demand'] * wlr for wlr in week_len_run ]           # значения - запас
))

df = df.withColumns(new_cols_dict).drop("demand", "stock")
print("Таблица по техническим неделям:")
df.show()


Продолжительность технических недель: [6, 7, 7, 7, 4]
Таблица по техническим неделям:
+----+---+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+
|prod|loc|wk1-dem|wk2-dem|wk3-dem|wk4-dem|wk5-dem|op-stk|wk1-stk|wk2-stk|wk3-stk|wk4-stk|wk5-stk|
+----+---+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+
|   1|  1|    600|    700|    700|    700|    400|  1000|    400|   -300|  -1000|  -1700|  -2100|
|   1|  2|    660|    770|    770|    770|    440|   400|   -260|  -1030|  -1800|  -2570|  -3010|
|   2|  1|    720|    840|    840|    840|    480|   300|   -420|  -1260|  -2100|  -2940|  -3420|
|   2|  2|    540|    630|    630|    630|    360|   250|   -290|   -920|  -1550|  -2180|  -2540|
+----+---+-------+-------+-------+-------+-------+------+-------+-------+-------+-------+-------+

